## 파인콘 백터 DB 저장
A × ( B / C )

- **A**: 전세사기피해주택의 임대인이 체납한 고지 또는 신고 건별 국세 금액  
- **B**: 전세사기피해주택의 가격  
- **C**: 전세사기피해주택의 임대인이 보유한 모든 주택의 가격 합계액

In [10]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain.vectorstores import Pinecone
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


## 1. 문서 내용 읽고 분할
loader = Docx2txtLoader('law_2.docx')

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 200,
)

document_list = loader.load_and_split(text_splitter=text_splitter)

## 2. 임베딩 -> 벡터 데이터베이스에 저장
## 2.1 환경변수 읽어오기
load_dotenv()

## 2.2 임베딩 모델 지정
embedding = OpenAIEmbeddings(model = 'text-embedding-3-large')

## 2.3 벡터 데이터베이스에 저장
## pinecone 저장
# database = PineconeVectorStore.from_documents(
#     embedding=embedding,
#     index_name = 'law-2',
#     documents= document_list,
# )

## pinecone 불러오기
api_key = os.getenv('PINECONE_API_KEY')
pc = Pinecone(api_key=api_key)

database = PineconeVectorStore(
    index=pc.Index('law-2'),
    embedding=embedding,
)

## 3. 질문 -> 벡터 데이터베이스(vector store)에서 유사도 검색
## 3.1 사용자 질문
query = '전세사기 피해자 금융지원에 대해 설명해주세요.'

## 3.2 벡터 데이터베이스에서 유사도 검색
retrieved_docs = database.similarity_search(query=query)

## 4. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달
## 4.1 프롬프트 작성
prompt = '''
[identity]
-당신은 전세사기 피해 법률 전문가입니다.
-[context]를 참고하여 사용자의 질문에 답변해주세요.

[context]
{retrieved_docs}

Question: {query}
'''

## 4.2 프롬프트 변수에 값 설정
formatted = prompt.format(retrieved_docs=retrieved_docs, query=query)

## 4.3  LLM 모델 생성(hatOpenAI인스턴스 생성)
llm = ChatOpenAI(model = 'gpt-4o')

## 4.4 LLM 모델에 질문과 검색된 문서 보내기
ai_message = llm.invoke(formatted)
ai_message

TypeError: Pinecone.__init__() got an unexpected keyword argument 'api_key'